In [10]:
import os
import sys

import numpy as np
import pandas as pd

from brainiak.funcalign.srm import SRM
# from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, LeaveOneOut
from sklearn.linear_model import RidgeClassifier
from sklearn.preprocessing import normalize
%autosave 5 
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set(style='white', context='talk', font_scale=1, rc={"lines.linewidth": 2})

Autosaving every 5 seconds


In [11]:
"""helpers
"""

def data_gen(signal, noise, n_TRs, n_voxels, n_subjects, signal_g):
    """Generate simulated data
    assuming the i-th subject in g-th group is the follwing...
        X_ig = S + S_g + E_ig, where...
            S is the shared response across all participants
            S_g is the shared response across all participants in group g
            E_ig is is individual specific ("noise")

    Parameters
    ----------
    signal : type
        Description of parameter `signal`.
    noise : type
        Description of parameter `noise`.
    n_TRs : type
        Description of parameter `n_TRs`.
    n_voxels : type
        Description of parameter `n_voxels`.
    n_subjects : type
        Description of parameter `n_subjects`.
    n_groups : type
        Description of parameter `n_groups`.
    signal_g : type
        Description of parameter `signal_g`.

    Returns
    -------
    type
        Description of returned object.

    """
    n_groups = len(signal_g)
    # shared signal
    S = np.random.normal(size=(n_TRs, n_voxels)) * signal
    Sg = [np.random.normal(size=(n_TRs, n_voxels)) * signal_g[i]
          for i in range(n_groups)]
    # each subject is some shared signal plus some noise
    X_all = np.zeros((n_groups, n_subjs_per_group, n_TRs, n_voxels))
    for g in range(n_groups):
        for i in range(n_subjects):
            E = np.random.normal(size=(n_TRs, n_voxels)) * noise
            X_all[g,i,:,:] = S + Sg[g] + E
    return X_all, S, Sg


def transform(srm_training_set, srm_test_set, n_components):
    # reformat the data from sklearn form to srm form
    transpose_op = [0, 2, 1]
    srm_training_set = np.transpose(srm_training_set, transpose_op)
    srm_test_set = np.transpose(srm_test_set, transpose_op)
    # fit SRM
    srm = SRM(features=n_components)
    srm.fit(srm_training_set)
    srm_training_set_transformed = srm.transform(srm_training_set)
    srm_test_set_transformed = srm.transform(srm_test_set)
    return srm_training_set_transformed, srm_test_set_transformed, srm


def compute_residual(X_i, W_i, S_i):
    """residual_i = X_i - W_i W_i.T X_i = X_i - W S_i
    *need to transpose the last term because brainiak's awkard format...

    Parameters
    ----------
    X_i : type
        Description of parameter `X_i`.
    W_i : type
        Description of parameter `W_i`.
    S_i : type
        Description of parameter `S_i`.

    Returns
    -------
    type
        Description of returned object.
    """
    residual = X_i - (W_i @ S_i).T
    return residual


In [12]:
"""sim params
"""
signal = 1
signal_g = [1, 1]
noise = 0
n_TRs = 128
n_voxels = 64
n_subjs_per_group = 4
n_components = 32
test_set_prop = .5  # this has to be .5 for now

#
n_groups = len(signal_g)
n_subjects = n_subjs_per_group * n_groups
group_assignments = np.repeat(np.arange(n_groups), n_subjs_per_group)

# gen data
X_all, S_all, S_g = data_gen(
    signal, noise, n_TRs, n_voxels, n_subjs_per_group, signal_g
)

print('data shape: (n_groups, n_subjs_per_group, n_TRs, n_voxels) = ', np.shape(X_all))
print('group_assignments', group_assignments)


data shape: (n_groups, n_subjs_per_group, n_TRs, n_voxels) =  (2, 4, 128, 64)
group_assignments [0 0 0 0 1 1 1 1]


In [13]:
"""set up training-test split
"""
test_set_size = int(n_TRs * test_set_prop)
train_set_size = n_TRs - test_set_size
test_id = np.arange(test_set_size)
train_id = np.arange(test_set_size, n_TRs)

# split the data
X_all_train = np.zeros((n_groups, n_subjs_per_group, train_set_size, n_voxels))
X_all_test = np.zeros((n_groups, n_subjs_per_group, test_set_size, n_voxels))
for g in range(n_groups):
    for i in range(n_subjs_per_group):
        X_all_train[g,i,:,:] = X_all[g, i, train_id, :]
        X_all_test[g,i,:,:] = X_all[g, i, test_id, :]

print(test_id)
print(train_id)
print(np.shape(X_all_train))
print(np.shape(X_all_test))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
[ 64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81
  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99
 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127]
(2, 4, 64, 64)
(2, 4, 64, 64)


In [14]:
"""srm transform"""
# run SRM for all subjects
S_all_train, S_all_test, srm_all = transform(
    np.reshape(X_all_train, (n_subjects, train_set_size, n_voxels)),
    np.reshape(X_all_test, (n_subjects, test_set_size, n_voxels)),
    n_components
)

# re-format the results 
S_all_train = np.reshape(
    S_all_train, 
    newshape=(n_groups, n_subjs_per_group, n_components, test_set_size)
)
S_all_test = np.reshape(
    S_all_test, 
    newshape=(n_groups, n_subjs_per_group, n_components, test_set_size)
)
srm_all_W = np.reshape(
    srm_all.w_, 
    newshape=(n_groups, n_subjs_per_group, n_voxels, n_components)
)

print(np.shape(S_all_train))
print(np.shape(S_all_test))
print(np.shape(srm_all_W))

(2, 4, 32, 64)
(2, 4, 32, 64)
(2, 4, 64, 32)


In [15]:
# comput residuals
R_all_train = np.zeros((n_groups, n_subjs_per_group, train_set_size, n_voxels))
R_all_test = np.zeros((n_groups, n_subjs_per_group, test_set_size, n_voxels))
for g in range(n_groups):
    for i in range(n_subjs_per_group):
        R_all_train[g][i] = compute_residual(
            X_all_train[g][i], srm_all_W[g][i], S_all_train[g][i])
        R_all_test[g][i] = compute_residual(
            X_all_test[g][i], srm_all_W[g][i], S_all_test[g][i])

print(np.shape(R_all_train))
print(np.shape(R_all_test))

(2, 4, 64, 64)
(2, 4, 64, 64)


In [16]:
# reformat the data set to vector (svm-readable) form 
n_features_native = test_set_size * n_voxels
n_features_shared = test_set_size * n_components
X_all_train_ = np.zeros((n_subjects, n_features_native))
X_all_test_ = np.zeros((n_subjects, n_features_native))
R_all_train_ = np.zeros((n_subjects, n_features_native))
R_all_test_ = np.zeros((n_subjects, n_features_native))

# 
for g in range(n_groups):
    subj_ids = np.arange(g * n_subjs_per_group, (g+1) * n_subjs_per_group)
    print(subj_ids)
    for i in range(n_subjs_per_group):
        X_all_train_[subj_ids[i]] = np.ravel(X_all_train[g][i])
        X_all_test_[subj_ids[i]] = np.ravel(X_all_test[g][i])
        R_all_train_[subj_ids[i]] = np.ravel(R_all_train[g][i])
        R_all_test_[subj_ids[i]] = np.ravel(R_all_test[g][i])
# normalize the data 
X_all_train_ = normalize(X_all_train_, norm='l2')
X_all_test_ = normalize(X_all_test_, norm='l2')
R_all_train_ = normalize(R_all_train_, norm='l2')
R_all_test_ = normalize(R_all_test_, norm='l2')
        
print(np.shape(X_all_train))
print(np.shape(X_all_test))

print(np.shape(X_all_train_))
print(np.shape(X_all_test_))
print(np.shape(R_all_train_))
print(np.shape(R_all_test_))

[0 1 2 3]
[4 5 6 7]
(2, 4, 64, 64)
(2, 4, 64, 64)
(8, 4096)
(8, 4096)
(8, 4096)
(8, 4096)


In [17]:
def fit_ridge_regression(data_train, data_test, label_test, label_train): 
    gs_ridge = GridSearchCV(
        RidgeClassifier(tol=tol, max_iter=max_iter, ),
        parameters,
        cv=LeaveOneOut(),
        return_train_score=True
    )
    gs_ridge.fit(data_train, label_train)
    gs_ridge_score = gs_ridge.score(data_test, label_test)
    return gs_ridge_score, gs_ridge

"""fit ridge regression"""

parameters = {'alpha':np.logspace(-2,2,num=5)}
tol = 1e-9
max_iter = 3000

gs_ridge_score, gs_ridge = fit_ridge_regression(
    X_all_train_, X_all_test_, group_assignments, group_assignments)
print(gs_ridge_score)

gs_ridge_score, gs_ridge = fit_ridge_regression(
    R_all_train_, R_all_test_, group_assignments, group_assignments)
print(gs_ridge_score)

0.5
0.5


In [18]:
pd.DataFrame(gs_ridge.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,mean_train_score,std_train_score
0,0.008783,0.018010,0.000402,0.000005,0.01,{'alpha': 0.01},1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0
1,0.001930,0.000010,0.000402,0.000007,0.1,{'alpha': 0.1},1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0
2,0.008678,0.017841,0.000415,0.000034,1,{'alpha': 1.0},1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0
3,0.001991,0.000153,0.000400,0.000015,10,{'alpha': 10.0},1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0
4,0.008172,0.017768,0.000314,0.000111,100,{'alpha': 100.0},0.0,0.0,0.0,0.0,...,0.571429,0.571429,0.571429,0.571429,0.571429,0.571429,0.571429,0.571429,0.571429,0.0
